In [7]:
# import python packages
import pandas as pd
import numpy as np

# import src code
import ase_evaluation as ae
#import benchmark_posteriors as bp
import bootstrap as btstrp
import cross_validation as cv
import logistic_regression as lr
import naive_bayes as nb
import network as ntwk
import process as prcs
#import RIVER as rvr
import simulate_data as sim
import benchmark_posteriors as bnchmk
import RIVER as river
import sklearn
from sklearn import metrics

In [2]:
s = sim.SimulateData("./test_output/", 'with_transfer', 0.4, 0.6, 0.01)
s._run()

# create a process object
p = prcs.Process('./test_output', 0.1)
p._process_simulated_data()

In [ ]:
p.tissues

In [ ]:
# create a bootstrap object
b = btstrp.Bootstrap(p.train_list, p.tissues, p.genomic_features, num_simulations=2, num_folds=3)
lambda_hp_children_dict, lambda_hp_parent = b._run_bootstrap()

In [ ]:
"tissue-specific transfer factors: ", lambda_hp_children_dict, "global transfer factor: ", lambda_hp_parent

In [9]:
n = ntwk.Network(train_list, test_list, p.tissues, p.genomic_features, 
                 with_transfer=True, output_dir="SPEER_output", 
                 lambda_hp_parent = None,
                 lambda_hp_children_dict = None,
                 e_distribution = 'cat')
train_list, test_list, beta_parent, beta_children, phi = n.run()

In [ ]:
train_list[3].head(10)

In [ ]:
lambda_hp_children_dict = {'brain': 0.01, 'group1': 0.01, 'muscle': 0.01, 'epithelial': 0.01, 'digestive': 0.01}
n = ntwk.Network(train_list, test_list, p.tissues, p.genomic_features, 
                 with_transfer=False, output_dir="SPEER_output", 
                 lambda_hp_parent = None, 
                 lambda_hp_children_dict = lambda_hp_children_dict, 
                 e_distribution = 'cat')
train_list, test_list, beta_parent, beta_children, phi = n.run()

In [4]:
n = river.River(p.train_list, p.test_list, p.genomic_features, output_dir='RIVER_output')
train_list, test_list, beta_parent_river, beta_children_river, phi_river = n.run()

EM iter:  0
EM iter:  1
EM iter:  2
EM iter:  3
EM iter:  4
EM iter:  5
EM iter:  6
EM iter:  7
EM iter:  8
EM iter:  9
EM iter:  10
EM iter:  11
EM iter:  12
EM iter:  13
EM iter:  14
EM iter:  15
EM iter:  16
EM iter:  17
EM iter:  18
EM iter:  19
EM iter:  20
EM iter:  21
EM iter:  22
EM iter:  23
EM iter:  24
EM iter:  25
EM iter:  26
EM iter:  27
EM iter:  28
EM iter:  29
EM iter:  30
EM iter:  31
EM iter:  32
EM iter:  33
EM iter:  34
EM iter:  35
EM iter:  36
EM iter:  37
EM iter:  38
EM iter:  39
EM iter:  40
EM iter:  41
EM iter:  42
EM iter:  43
EM iter:  44
EM iter:  45
EM iter:  46
EM iter:  47
EM iter:  48
EM iter:  49
EM iter:  50
EM iter:  51
EM iter:  52
EM iter:  53
EM iter:  54
EM iter:  55
EM iter:  56
EM iter:  57
EM iter:  58
EM iter:  59
EM iter:  60
EM iter:  61
EM iter:  62
EM iter:  63
EM iter:  64
EM iter:  65
EM iter:  66
EM iter:  67
EM iter:  68
EM iter:  69
EM iter:  70
EM iter:  71
EM iter:  72
EM iter:  73
EM iter:  74
EM iter:  75
EM iter:  76
EM iter: 

In [15]:
sklearn.metrics.roc_auc_score(train_list[4]["z_label"], train_list[4]["RIVER"])

0.75004561211457765

In [ ]:
bn = bnchmk.BenchmarkPosteriors(train_list, test_list, p.genomic_features)
train_list, test_list = bn.fit_models()

In [ ]:
train_list[0].head()

In [ ]:
import sklearn
from sklearn import metrics
for i in range(5):
    print(sklearn.metrics.roc_auc_score(test_list[i]["z_label"], test_list[i]["SPEER"]),
          sklearn.metrics.roc_auc_score(test_list[i]["z_label"], test_list[i]["SPEER without transfer"]),
          sklearn.metrics.roc_auc_score(test_list[i]["z_label"], test_list[i]["tissue specific genome only"]),
          sklearn.metrics.roc_auc_score(test_list[i]["z_label"], test_list[i]["shared tissue genome only"]))

In [ ]:
sklearn.metrics.roc_auc_score(test_list[0]["z_labels"], test_list[0]["SPEER without transfer"])

In [ ]:
bn = bnchmk.BenchmarkPosteriors(p.train_list, p.test_list, p.genomic_features)